# 第一个应用

```{note}
我们用一个简单的示例来说明Spark Python API的基本用法。
```

## 数据

数据集是一个csv文件，共三列，分别表示州名、彩虹糖的颜色、彩虹糖的数量

In [1]:
!head ../data/mnm_dataset.csv

## 创建SparkSession

In [2]:
import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import count

In [3]:
# 加括号是为了方便换行
spark = (SparkSession
         .builder
         # appName表示此应用的名字
         .appName("PythonMnMCount")
         .getOrCreate())

## 创建DataFrame

就像使用pandas的DataFrame一样。

In [4]:
# 指定文件类型为csv
df = (spark.read.format("csv")
      # csv有header
      .option("header", "true")
      # 自动推测字段类型
      .option("inferSchema", "true")
      # 文件地址
      .load("../data/mnm_dataset.csv"))

## 统计各州各颜色糖的总数

In [5]:
count_df = (df
            # 选取三列
            .select("State", "Color", "Count")
            # 各州、各颜色
            .groupBy("State", "Color")
            # 数量加起来，成为新的一列`Total`
            .agg(count("Count").alias("Total"))
            # 按`Total`降序排列
            .orderBy("Total", ascending=False))

In [6]:
# 展示前10行
# truncate: 是否每行最多只显示20个字符，默认为true。 
count_df.show(n=10, truncate=False)

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|CA   |Yellow|1807 |
|WA   |Green |1779 |
|OR   |Orange|1743 |
|TX   |Green |1737 |
|TX   |Red   |1725 |
|CA   |Green |1723 |
|CO   |Yellow|1721 |
|CA   |Brown |1718 |
|CO   |Green |1713 |
|NV   |Orange|1712 |
+-----+------+-----+
only showing top 10 rows



## 只统计加州

In [7]:
ca_count_df = (df
               .select("State", "Color", "Count")
               # 通过where只统计加州
               .where(df.State == "CA")
               .groupBy("State", "Color")
               .agg(count("Count").alias("Total"))
               .orderBy("Total", ascending=False))

In [8]:
ca_count_df.show(n=10, truncate=False)

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|CA   |Yellow|1807 |
|CA   |Green |1723 |
|CA   |Brown |1718 |
|CA   |Orange|1657 |
|CA   |Red   |1656 |
|CA   |Blue  |1603 |
+-----+------+-----+

